In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict
from dotenv import load_dotenv

In [2]:
load_dotenv()

llm = ChatOpenAI()

class QuoteState(TypedDict):
    topic: str
    quote: str
    summary: str


In [8]:

def make_quote(state: QuoteState):
    prompt = f"Write a short inspirational quote about {state['topic']}."
    quote = llm.invoke(prompt).content
    return {"quote": quote}

def summarize_quote(state: QuoteState):
    prompt = f"Summarize this quote {state['quote']}"
    summary = llm.invoke(prompt).content
    return {"summary": summary}


In [4]:
graph = StateGraph(QuoteState)
graph.add_node("make_quote", make_quote)
graph.add_node("summarize_quote", summarize_quote)

graph.add_edge(START, "make_quote")
graph.add_edge("make_quote", "summarize_quote")
graph.add_edge("summarize_quote", END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)



In [10]:
config = {"configurable": {"thread_id": "1"}}

result = workflow.invoke({"topic": "Warren Buffett"}, config=config)

In [11]:
print("Quote:", result["quote"])


Quote: "Warren Buffett's success is a testament to the power of patience, wisdom, and discipline in the world of investing."


In [12]:
print("Summary:", result["summary"])

Summary: Patience, wisdom, discipline lead to investing success.


In [14]:
workflow.get_state(config)


StateSnapshot(values={'topic': 'Warren Buffett', 'quote': '"Warren Buffett\'s success is a testament to the power of patience, wisdom, and discipline in the world of investing."', 'summary': 'Patience, wisdom, discipline lead to investing success.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0989fc-eb65-6798-8006-a0d107c65b8d'}}, metadata={'source': 'loop', 'step': 6, 'parents': {}}, created_at='2025-09-23T17:07:38.376386+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0989fc-dc1a-69e4-8005-9de3444fca82'}}, tasks=(), interrupts=())